In [5]:
import os
import numpy as np
import nibabel as nib
import cv2
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.ndimage import gaussian_filter1d
import math
import gzip
import shutil
import tempfile
from skimage.filters import sobel
from skimage.feature import canny
import matplotlib
matplotlib.use('TkAgg')  # or 'Qt5Agg', 'GTK3Agg', etc. depending on your system

# Helper functions
def handle_compressed_file(filepath):
    """Handle both .nii.gz and .niigz files"""
    if filepath.endswith('.niigz'):
        with tempfile.NamedTemporaryFile(suffix='.nii.gz', delete=False) as tmp_file:
            with gzip.open(filepath, 'rb') as f_in:
                shutil.copyfileobj(f_in, tmp_file)
            return tmp_file.name
    return filepath

def rotate_image_clockwise(image, angle=90):
    """Rotate image clockwise with better boundary handling"""
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, -angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_NEAREST, borderMode=cv2.BORDER_REPLICATE)
    return rotated

def enhanced_edge_detection(seg_binary):
    """Improved edge detection using multiple methods"""
    seg_binary = (seg_binary > 0).astype(np.uint8) * 255
    edges_canny = canny(seg_binary/255.0, sigma=1.5)
    edges_sobel = sobel(seg_binary/255.0) > 0.1
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    gradient = cv2.morphologyEx(seg_binary, cv2.MORPH_GRADIENT, kernel)
    combined_edges = (edges_canny | edges_sobel) | (gradient > 0)
    combined_edges = combined_edges.astype(np.uint8) * 255
    kernel = np.ones((3,3), np.uint8)
    combined_edges = cv2.morphologyEx(combined_edges, cv2.MORPH_CLOSE, kernel)
    return combined_edges

def detect_landmarks(seg_binary, delta_y_fraction=0.15, y_tolerance=20, curvature_threshold=0.2, min_x_separation=0.2):
    """Enhanced landmark detection combining view-specific and flexible logic."""
    seg_binary = (seg_binary > 0).astype(np.uint8) * 255
    contours, _ = cv2.findContours(seg_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    if not contours:
        raise ValueError("No LV contour found")
    contour = max(contours, key=cv2.contourArea).squeeze()
    x = contour[:, 0].astype(float)
    y = contour[:, 1].astype(float)
    img_width = seg_binary.shape[1]
    x_smooth = gaussian_filter1d(x, sigma=5, mode='wrap')
    y_smooth = gaussian_filter1d(y, sigma=5, mode='wrap')
    dx = np.gradient(x_smooth)
    dy = np.gradient(y_smooth)
    ddx = np.gradient(dx)
    ddy = np.gradient(dy)
    curvature = np.abs(dx * ddy - dy * ddx) / (dx**2 + dy**2 + 1e-10)**1.5
    curvature = (curvature - np.min(curvature)) / (np.max(curvature) - np.min(curvature) + 1e-10)
    apex_idx = np.argmin(y)
    apex_point = (int(x[apex_idx]), int(y[apex_idx]))
    mean_x = np.mean(x)
    
    # Determine view type
    if abs(np.median(x) - mean_x) < 0.3*img_width:
        view_type = '2CH'
    elif abs(np.median(x) - mean_x) < 0.5*img_width:
        view_type = '3CH'
    else:
        view_type = '4CH'
        
    max_y = max(y)
    min_y = min(y)
    basal_threshold = max_y - delta_y_fraction * (max_y - min_y)
    peaks, _ = find_peaks(curvature, prominence=0.0005)
    basal_peaks = [i for i in peaks if y[i] >= basal_threshold]
    if len(basal_peaks) < 2:
        basal_threshold = max_y - 2 * delta_y_fraction * (max_y - min_y)
        basal_peaks = [i for i in peaks if y[i] >= basal_threshold]
    basal_points = [i for i in range(len(y)) if y[i] >= basal_threshold]
    
    if view_type == '4CH':
        if basal_points:
            sorted_by_y = sorted(basal_points, key=lambda i: -y[i])
            lateral_idx = sorted_by_y[0]
            mean_x = np.mean(x[basal_points])
            septal_candidates = [i for i in basal_points 
                                 if (x[i] - mean_x) * (x[lateral_idx] - mean_x) < 0
                                 and curvature[i] >= curvature_threshold]
            if septal_candidates:
                septal_idx = max(septal_candidates, key=lambda i: curvature[i])
            else:
                other_points = [i for i in basal_points if i != lateral_idx]
                if other_points:
                    septal_idx = max(other_points, key=lambda i: curvature[i])
                else:
                    septal_idx = lateral_idx
        else:
            raise ValueError("No basal points found for 4CH")
    elif view_type == '3CH':
        # For 3CH view, we need to identify anterior and inferior points
        if basal_points:
            # Find the two most basal points with highest curvature
            basal_peaks_sorted = sorted(basal_peaks, key=lambda i: curvature[i], reverse=True)
            if len(basal_peaks_sorted) >= 2:
                anterior_idx = basal_peaks_sorted[0]
                inferior_idx = basal_peaks_sorted[1]
            else:
                # Fallback: use leftmost and rightmost basal points
                left_idx = basal_points[np.argmin(x[basal_points])]
                right_idx = basal_points[np.argmax(x[basal_points])]
                if curvature[left_idx] > curvature[right_idx]:
                    anterior_idx = left_idx
                    inferior_idx = right_idx
                else:
                    anterior_idx = right_idx
                    inferior_idx = left_idx
            septal_point = (int(x[anterior_idx]), int(y[anterior_idx]))
            lateral_point = (int(x[inferior_idx]), int(y[inferior_idx]))
        else:
            raise ValueError("No basal points found for 3CH")
    else:  # 2CH
        if len(basal_peaks) >= 2:
            basal_peaks_sorted = sorted(basal_peaks, key=lambda i: curvature[i], reverse=True)
            septal_idx = basal_peaks_sorted[0]
            septal_side = "right" if x[septal_idx] > mean_x else "left"
            lateral_candidates = [i for i in basal_peaks_sorted[1:] 
                                  if ((x[i] > mean_x and septal_side == "left") or 
                                      (x[i] < mean_x and septal_side == "right")) 
                                  and curvature[i] >= curvature_threshold / 2]
            if lateral_candidates:
                scores = []
                for i in lateral_candidates:
                    curvature_score = curvature[i] * 0.6
                    y_alignment_score = (1 - min(1, abs(y[i] - y[septal_idx]) / y_tolerance)) * 0.4
                    scores.append(curvature_score + y_alignment_score)
                lateral_idx = lateral_candidates[np.argmax(scores)]
            else:
                lateral_idx = basal_peaks_sorted[1]
            min_x_dist = min_x_separation * img_width
            if abs(x[septal_idx] - x[lateral_idx]) < min_x_dist:
                outward_dir = 1 if x[lateral_idx] > mean_x else -1
                for i in basal_peaks_sorted[2:]:
                    if abs(x[i] - x[septal_idx]) > min_x_dist and \
                       ((x[i] > mean_x and outward_dir > 0) or (x[i] < mean_x and outward_dir < 0)):
                        lateral_idx = i
                        break
            if abs(x[septal_idx] - x[lateral_idx]) < min_x_dist:
                print(f"Warning: Points still close after fallback. Distance: {abs(x[septal_idx] - x[lateral_idx]):.1f} px")
        else:
            if basal_points:
                left_idx = basal_points[np.argmin(x[basal_points])]
                right_idx = basal_points[np.argmax(x[basal_points])]
                if curvature[left_idx] > curvature[right_idx]:
                    septal_idx = left_idx
                    lateral_idx = right_idx
                else:
                    septal_idx = right_idx
                    lateral_idx = left_idx
            else:
                raise ValueError("No basal points found for 2CH")
    
    if view_type != '3CH':  # Already set for 3CH
        septal_point = (int(x[septal_idx]), int(y[septal_idx]))
        lateral_point = (int(x[lateral_idx]), int(y[lateral_idx]))
    
    mitral_mid = ((septal_point[0] + lateral_point[0]) / 2, (septal_point[1] + lateral_point[1]) / 2)
    print(f"View type: {view_type}")
    print(f"Apex: {apex_point}")
    print(f"Septal/Anterior: {septal_point}")
    print(f"Lateral/Inferior: {lateral_point}")
    print(f"Mitral Mid: {mitral_mid}")
    return apex_point, septal_point, lateral_point, mitral_mid, curvature, contour, view_type

# True Simpson's Biplane Method Implementation
def compute_simpson_biplane_diameters(seg_binary, apex, mitral_mid, pixel_spacing=(0.1, 0.1), nr_disks=20):
    """
    True Simpson's method: Divides LV into parallel disks perpendicular to long axis
    and measures diameters for each disk
    """
    # Create edge detection and filled mask
    edges = enhanced_edge_detection(seg_binary)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    if not contours:
        return 0.0, [], [], 0.0, None, None
    
    max_contour = max(contours, key=cv2.contourArea)
    filled_mask = np.zeros_like(seg_binary, dtype=np.uint8)
    cv2.drawContours(filled_mask, [max_contour], -1, 1, thickness=cv2.FILLED)
    
    # Create boundary from filled mask
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    boundary = cv2.morphologyEx(filled_mask, cv2.MORPH_GRADIENT, kernel)
    
    # Long axis vector (apex to mitral_mid)
    dx_long = mitral_mid[0] - apex[0]
    dy_long = mitral_mid[1] - apex[1]
    L_cm = np.sqrt((dx_long * pixel_spacing[0])**2 + (dy_long * pixel_spacing[1])**2)
    if L_cm == 0:
        return L_cm, [], [], 0.0, None, None
    
    # Normalize long axis vector
    long_norm = np.sqrt(dx_long**2 + dy_long**2) + 1e-10
    long_vector = np.array([dx_long/long_norm, dy_long/long_norm])
    
    # Perpendicular vector (rotated 90°)
    perp_vector = np.array([-long_vector[1], long_vector[0]])
    
    max_dist = max(seg_binary.shape) // 2
    diameters_cm = []
    disk_info = []
    
    # Calculate disk height
    h_cm = L_cm / nr_disks
    
    for i in range(nr_disks):
        # Disk center along long axis
        t = (i + 0.5) / nr_disks
        center_x = apex[0] + t * dx_long
        center_y = apex[1] + t * dy_long
        
        # Find boundary intersections along perpendicular direction
        point1 = point2 = None
        
        # Search in negative perpendicular direction
        for dist in range(1, max_dist):
            px = int(round(center_x - dist * perp_vector[0]))
            py = int(round(center_y - dist * perp_vector[1]))
            
            if not (0 <= px < seg_binary.shape[1] and 0 <= py < seg_binary.shape[0]):
                break
            
            if boundary[py, px]:
                point1 = (px, py)
                break
        
        # Search in positive perpendicular direction
        for dist in range(1, max_dist):
            px = int(round(center_x + dist * perp_vector[0]))
            py = int(round(center_y + dist * perp_vector[1]))
            
            if not (0 <= px < seg_binary.shape[1] and 0 <= py < seg_binary.shape[0]):
                break
            
            if boundary[py, px]:
                point2 = (px, py)
                break
        
        if point1 and point2:
            # Calculate diameter in cm
            dx_px = point2[0] - point1[0]
            dy_px = point2[1] - point1[1]
            diameter_cm = np.sqrt((dx_px * pixel_spacing[0])**2 + (dy_px * pixel_spacing[1])**2)
            diameters_cm.append(diameter_cm)
        else:
            diameters_cm.append(0.0)
        
        disk_info.append({
            'center': (center_x, center_y),
            'point1': point1,
            'point2': point2,
            'diameter_cm': diameters_cm[-1] if diameters_cm else 0.0
        })
    
    return L_cm, diameters_cm, disk_info, h_cm, None, filled_mask

def compute_simpson_biplane_volume(L4_cm, L2_cm, diameters4_cm, diameters2_cm, h_cm_4ch, h_cm_2ch, nr_disks=20):
    """
    True Simpson's biplane volume calculation using elliptical disks
    V = (π/4) * Σ [d1_i * d2_i] * h
    where h is the average disk height from both views
    """
    # Validate inputs
    if nr_disks <= 0:
        return 0.0, [], 0.0
    
    # Use average disk height
    h_avg_cm = (h_cm_4ch + h_cm_2ch) / 2
    
    total_volume = 0.0
    disk_volumes = []
    
    # Ensure both diameters lists have same length
    min_len = min(len(diameters4_cm), len(diameters2_cm))
    
    for i in range(min_len):
        d1_i = diameters4_cm[i]
        d2_i = diameters2_cm[i]
        
        # Skip invalid diameters
        if d1_i < 0.1 or d2_i < 0.1:
            disk_volumes.append(0.0)
            continue
            
        # Simpson disk volume formula: elliptical area × height
        disk_volume = (np.pi / 4) * d1_i * d2_i * h_avg_cm
        total_volume += disk_volume
        disk_volumes.append(disk_volume)
    
    return total_volume, disk_volumes, 0.0

def visualize_processing(seg_data, landmarks, disk_info, view, phase, pixel_spacing):
    apex, septal, lateral, mitral_mid = landmarks
    plt.figure(figsize=(18, 6))
    edges = enhanced_edge_detection(seg_data)
    plt.subplot(131)
    plt.imshow(edges, cmap='gray')
    plt.title(f'{view} {phase} - Edge Detection')
    plt.subplot(132)
    plt.imshow(seg_data, cmap='gray')
    plt.scatter(apex[0], apex[1], c='red', s=100, marker='x', label='Apex')
    plt.scatter(septal[0], septal[1], c='blue', s=100, marker='x', label='Septal/Anterior')
    plt.scatter(lateral[0], lateral[1], c='green', s=100, marker='x', label='Lateral/Inferior')
    plt.scatter(mitral_mid[0], mitral_mid[1], c='magenta', s=150, marker='+', label='Mitral Mid')
    plt.plot([apex[0], mitral_mid[0]], [apex[1], mitral_mid[1]], 'y--', linewidth=2, label='Long Axis')
    plt.plot([septal[0], lateral[0]], [septal[1], lateral[1]], 'c--', linewidth=2, label='Mitral Plane')
    plt.legend(loc='upper right')
    plt.title(f'{view} {phase} - Landmarks')
    plt.subplot(133)
    plt.imshow(seg_data, cmap='gray')
    for disk in disk_info:
        if disk['point1'] and disk['point2']:
            plt.plot([disk['point1'][0], disk['point2'][0]], 
                     [disk['point1'][1], disk['point2'][1]], 
                     'r-', linewidth=1.5)
            plt.scatter(disk['center'][0], disk['center'][1], c='yellow', s=30)
    plt.scatter(apex[0], apex[1], c='red', s=100, marker='x')
    plt.scatter(septal[0], septal[1], c='blue', s=100, marker='x')
    plt.scatter(lateral[0], lateral[1], c='green', s=100, marker='x')
    plt.scatter(mitral_mid[0], mitral_mid[1], c='magenta', s=150, marker='+')
    plt.plot([apex[0], mitral_mid[0]], [apex[1], mitral_mid[1]], 'y--', linewidth=1)
    plt.plot([septal[0], lateral[0]], [septal[1], lateral[1]], 'c--', linewidth=1)
    plt.title(f'{view} {phase} - Simpson Method Disks')
    plt.tight_layout()
    plt.show()

def visualize_volume_results(ed_vol, es_vol, ef, ed_disk_volumes, es_disk_volumes, view_combination):
    """Visualize the volume calculation results"""
    plt.figure(figsize=(12, 6))
    
    # Volume distribution along long axis
    plt.subplot(121)
    plt.plot(ed_disk_volumes, 'b-', label='ED Volume')
    plt.plot(es_disk_volumes, 'r-', label='ES Volume')
    plt.xlabel('Disk Number')
    plt.ylabel('Volume Contribution (ml)')
    plt.title('Volume Distribution Along Long Axis')
    plt.legend()
    
    # Bar chart of EDV, ESV, and EF
    plt.subplot(122)
    plt.bar(['EDV', 'ESV', 'EF'], [ed_vol, es_vol, ef], 
           color=['blue', 'red', 'green'])
    plt.ylabel('Volume (ml) / Percentage (%)')
    plt.title(f'Simpson Biplane Method ({view_combination}): EF = {ef:.1f}%')
    
    # Add text annotations
    for i, v in enumerate([ed_vol, es_vol, ef]):
        plt.text(i, v + max([ed_vol, es_vol, ef])/20, f'{v:.1f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()

def process_patient(patient_dir, view_combination, visualize=False):
    patient_id = os.path.basename(patient_dir)
    print(f"\nProcessing Patient: {patient_id}")
    print(f"Using view combination: {view_combination}")
    
    # Initialize volumes dictionary based on selected views
    views = view_combination.split('+')
    volumes = {view: {'ED': None, 'ES': None} for view in views}
    all_landmarks = {}
    all_disk_info = {}
    
    try:
        for file in os.listdir(patient_dir):
            if not (file.endswith('_gt.niigz') or file.endswith('_gt.nii.gz')):
                continue
                
            view = None
            phase = None
            
            # Check for all view types
            if '4ch' in file.lower():
                view = '4CH'
            elif '3ch' in file.lower():
                view = '3CH'
            elif '2ch' in file.lower():
                view = '2CH'
                
            if 'ed' in file.lower():
                phase = 'ED'
            elif 'es' in file.lower():
                phase = 'ES'
                
            # Only process if view is in our selected combination
            if not view or not phase or view not in views:
                continue
                
            print(f"\nProcessing {view} {phase}...")
            filepath = os.path.join(patient_dir, file)
            temp_file = None
            
            try:
                if file.endswith('.niigz'):
                    temp_file = handle_compressed_file(filepath)
                    gt_img = nib.load(temp_file)
                else:
                    gt_img = nib.load(filepath)
                    
                gt_data = gt_img.get_fdata()
                pixel_spacing = gt_img.header.get_zooms()[:2]
                mask = np.zeros_like(gt_data, dtype=np.uint8)
                mask[gt_data == 1] = 255
                rotated_mask = rotate_image_clockwise(mask)
                
                try:
                    apex, septal, lateral, mitral_mid, _, _, detected_view = detect_landmarks(rotated_mask)
                    L_cm, diameters_cm, disk_info, h_cm, _, _ = compute_simpson_biplane_diameters(
                        rotated_mask, apex, mitral_mid, pixel_spacing
                    )
                    
                    volumes[view][phase] = {
                        'L_cm': L_cm,
                        'diameters_cm': diameters_cm,
                        'h_cm': h_cm,
                        'dead_area_cm2': 0.0,
                        'status': 'success'
                    }
                    
                    all_landmarks[(view, phase)] = (apex, septal, lateral, mitral_mid)
                    all_disk_info[(view, phase)] = disk_info
                    
                    if visualize:
                        visualize_processing(rotated_mask, (apex, septal, lateral, mitral_mid), 
                                             disk_info, view, phase, pixel_spacing)
                                             
                except Exception as landmark_error:
                    print(f"Landmark detection failed for {view} {phase}: {str(landmark_error)}")
                    volumes[view][phase] = {
                        'status': f'landmark_error: {str(landmark_error)}'
                    }
                    
            except Exception as file_error:
                print(f"Error processing file {file}: {str(file_error)}")
                if view and phase:
                    volumes[view][phase] = {
                        'status': f'file_error: {str(file_error)}'
                    }
            finally:
                if temp_file and os.path.exists(temp_file):
                    os.unlink(temp_file)
                    
    except Exception as patient_error:
        print(f"Error processing patient {patient_id}: {str(patient_error)}")
        return None, None, None
    
    # Check for complete data
    missing = []
    for view in views:
        for phase in ['ED', 'ES']:
            if not volumes[view][phase] or volumes[view][phase].get('status') != 'success':
                missing.append(f"{view} {phase}")
    
    if missing:
        print(f"\nIncomplete data for EF calculation. Missing: {', '.join(missing)}")
    else:
        try:
            # Calculate volumes using true Simpson biplane method
            ed_vol, ed_disk_volumes, _ = compute_simpson_biplane_volume(
                volumes[views[0]]['ED']['L_cm'],
                volumes[views[1]]['ED']['L_cm'],
                volumes[views[0]]['ED']['diameters_cm'],
                volumes[views[1]]['ED']['diameters_cm'],
                volumes[views[0]]['ED']['h_cm'],
                volumes[views[1]]['ED']['h_cm']
            )
            
            es_vol, es_disk_volumes, _ = compute_simpson_biplane_volume(
                volumes[views[0]]['ES']['L_cm'],
                volumes[views[1]]['ES']['L_cm'],
                volumes[views[0]]['ES']['diameters_cm'],
                volumes[views[1]]['ES']['diameters_cm'],
                volumes[views[0]]['ES']['h_cm'],
                volumes[views[1]]['ES']['h_cm']
            )
            
            ef = ((ed_vol - es_vol) / ed_vol) * 100 if ed_vol > 0 else 0
            
            # Store results
            volumes['EDV'] = ed_vol
            volumes['ESV'] = es_vol
            volumes['EF'] = ef
            volumes['ED_disk_volumes'] = ed_disk_volumes
            volumes['ES_disk_volumes'] = es_disk_volumes
            volumes['view_combination'] = view_combination
            
            print(f"\nSimpson Biplane Method Results ({view_combination}):")
            print(f"EDV: {ed_vol:.2f} ml")
            print(f"ESV: {es_vol:.2f} ml")
            print(f"EF: {ef:.2f}%")
            
            if visualize:
                visualize_volume_results(ed_vol, es_vol, ef, ed_disk_volumes, es_disk_volumes, view_combination)
                
        except Exception as vol_error:
            print(f"Volume calculation error: {str(vol_error)}")
    
    return volumes, all_landmarks, all_disk_info

if __name__ == "__main__":
    base_dir = r"E:\echo heart\Resources"
    patient_dirs = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
    
    if not patient_dirs:
        print("No patient directories found")
    else:
        print("Available patients:")
        for i, patient in enumerate(patient_dirs):
            print(f"{i+1}. {patient}")
            
        choice = input("Enter patient number to process: ")
        
        if choice.isdigit():
            idx = int(choice) - 1
            if 0 <= idx < len(patient_dirs):
                full_path = os.path.join(base_dir, patient_dirs[idx])
                
                # Ask for view combination
                print("\nAvailable view combinations:")
                print("1. 2CH + 4CH")
                print("2. 2CH + 3CH")
                print("3. 3CH + 4CH")
                
                view_choice = input("Select view combination (1-3): ")
                
                if view_choice == '1':
                    view_combination = '2CH+4CH'
                elif view_choice == '2':
                    view_combination = '2CH+3CH'
                elif view_choice == '3':
                    view_combination = '3CH+4CH'
                else:
                    print("Invalid choice, defaulting to 2CH+4CH")
                    view_combination = '2CH+4CH'
                
                visualize_choice = input("Visualize processing? (y/n): ").lower()
                visualize = visualize_choice == 'y'
                
                # Set matplotlib to interactive mode
                plt.ion()  # Turn on interactive mode
                
                process_patient(full_path, view_combination, visualize=visualize)
                
                # Keep plots open if in interactive mode
                if visualize:
                    plt.ioff()  # Turn off interactive mode
                    plt.show()  # Keep plots open until closed by user
            else:
                print("Invalid patient number")
        else:
            print("Invalid input")

Available patients:
1. patient0001
2. patient0002
3. patient0003
4. patient0004
5. patient0005
6. patient0006
7. patient0007
8. patient0008
9. patient0009
10. patient0010

Available view combinations:
1. 2CH + 4CH
2. 2CH + 3CH
3. 3CH + 4CH

Processing Patient: patient0004
Using view combination: 2CH+4CH

Processing 2CH ED...
View type: 2CH
Apex: (233, 155)
Septal/Anterior: (123, 367)
Lateral/Inferior: (228, 384)
Mitral Mid: (175.5, 375.5)

Processing 2CH ES...
View type: 2CH
Apex: (228, 175)
Septal/Anterior: (163, 345)
Lateral/Inferior: (230, 350)
Mitral Mid: (196.5, 347.5)

Processing 4CH ED...
View type: 2CH
Apex: (223, 154)
Septal/Anterior: (238, 396)
Lateral/Inferior: (126, 364)
Mitral Mid: (182.0, 380.0)

Processing 4CH ES...
View type: 2CH
Apex: (219, 155)
Septal/Anterior: (245, 364)
Lateral/Inferior: (145, 343)
Mitral Mid: (195.0, 353.5)

Simpson Biplane Method Results (2CH+4CH):
EDV: 99950.18 ml
ESV: 35997.75 ml
EF: 63.98%


: 